<a href="https://colab.research.google.com/github/ryali93/utilitarios/blob/master/extractData/extractdata_senamhi_historic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
e = open("est.json", "r")
data = e.read()
e.close()

import json
est = json.loads(data)
est

In [0]:
import pandas as pd

In [0]:
estaciones = pd.DataFrame(est)
estac = estaciones["codigo"]


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

def fechas_df(df, field_dates):
  fy = list(df[field_dates])[0]
  len_first_year = len(df[df[field_dates] == fy])
  date_rng_fy = pd.date_range(start='1/1/{}'.format(df[field_dates][0]), end='31/12/{}'.format(df[field_dates][0]), freq='D')
  dates_first_year = pd.date_range(start=date_rng_fy[-len_first_year], end='31/12/{}'.format(df[field_dates][0]), freq='D')

  ly = list(df[field_dates])[-1]
  len_last_year = len(df[df[field_dates] == ly])
  date_rng_ly = pd.date_range(start='1/1/{}'.format(ly), end='31/12/{}'.format(ly), freq='D')
  dates_last_year = pd.date_range(start='1/1/{}'.format(ly), end=date_rng_ly[len_last_year-1], freq='D')

  dates = pd.date_range(start=date_rng_fy[-len_first_year], end=date_rng_ly[len_last_year-1], freq='D')
  df[field_dates] = dates
  return df

def download_historic_senamhi(codigo):
  response = requests.get("https://web2.senamhi.gob.pe/descarga/?cod={}".format(codigo))
  soup = BeautifulSoup(response.text, "html.parser")

  lista = []
  for s in soup.find_all('script', {"type":"text/javascript"}):
      lista.append(s.text)

  m = lista[1].replace("\n", "").replace("\r", "").replace("\t", "").replace("null", "None")
  datos = re.findall("\[.*?]", m)
  t = [d for d in datos if len(d)>100]

  # Fechas
  d = re.findall("(categories: \[.*?])", t[0])[0]
  d = d.split(":")[1][2:-2]
  fechas_lista = [int(f) for f in d.replace("'", "").split(",")]
  fechas_lista

  # Precipitacion
  p = re.findall("(data: \[.*?])", t[1])[0]
  p = p.split(":")[1][2:-2]

  pp_lista = []
  for pp in p.split(","):
    if pp == "None":
      pp_lista.append(None)
    else:
      pp_lista.append(float(pp))

  # Temp Max
  temp = t[2][1:-2]
  temp_max_lista = []
  for temp in temp.split(","):
    if temp == "None":
      temp_max_lista.append(None)
    else:
      temp_max_lista.append(float(temp))

  # Temp Min
  temp = t[3][1:-2]
  temp_min_lista = []
  for temp in temp.split(","):
    if temp == "None":
      temp_min_lista.append(None)
    else:
      temp_min_lista.append(float(temp))

  dicc = {
      "fechas": fechas_lista,
      "pp": pp_lista,
      "temp_max": temp_max_lista,
      "temp_min": temp_min_lista,
  }

  df = pd.DataFrame(dicc)
  df = fechas_df(df, "fechas")
  df.to_csv("drive/My Drive/SENAMHI/historical_data/{}.csv".format(codigo), index=False)


In [0]:
download_historic_senamhi(152204)

In [85]:
error_log = []
for e in estac:
  try:
    print(e)
    download_historic_senamhi(e)
  except:
    error_log.append(e)

152204
152210
155105
000441
150903
154111
000435
154110
154107
150902
000677
157317
000765
158208
000755
000864
157309
000749
000837
000801
007308
157314
157312
157313
157310
000833
000830
000746
000804
000805
157315
000754
000758
157311
000849
000847
158209
000839
157300
157329
000736
000743
157223
156211
156212
000745
157206
151602
157200
000352
153109
153108
153208
000349
000250
000241
153107
152129
153103
000341
153235
000353
153223
000373
000369
000393
000304
153201
153203
000260
000240
000370
153111
153110
000395
000607
000687
000812
000844
000686
000809
156306
000690
156126
000659
000657
151503
156114
156130
157101
000647
156123
156122
000625
006200
000457
000404
000552
000468
000469
000700
000650
157102
000698
000730
000640
000791
156113
000727
000477
000608
155231
155229
155115
000635
155121
000503
154101
153101
153206
000374
154103
154112
154102
154106
153327
000325
000332
000320
153102
000333
152107
000301
153114
000340
151208
000547
151204
000543
151209
000618
155201
000541

In [87]:
len(error_log)

16

In [91]:
len(estac)

293

In [92]:
error_log

['157312',
 '153235',
 '000809',
 '156130',
 '156100',
 '000539',
 '153315',
 '000840',
 '000783',
 '000776',
 '000764',
 '153300',
 '153319',
 '153322',
 '000875',
 '158323']